In [1]:
import functools
import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL="https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
train_file_path=tf.keras.utils.get_file("train.csv",TRAIN_DATA_URL)
test_file_path=tf.keras.utils.get_file("eval.csv",TEST_DATA_URL)

In [3]:
np.set_printoptions(precision=3,suppress=True)

In [4]:
LABEL_COLUMN='survived'
LABELS=[0,1]

In [5]:
def get_dataset(file_path, **kwargs):
    dataset=tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs
    )
    return dataset
raw_train_data=get_dataset(train_file_path)
raw_test_data=get_dataset(test_file_path)

In [6]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

In [7]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'female']
age                 : [11. 19. 36. 28. 40.]
n_siblings_spouses  : [5 0 1 0 0]
parch               : [2 0 2 0 0]
fare                : [46.9   10.5   27.75   8.663 13.   ]
class               : [b'Third' b'Second' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'n' b'y' b'y']


In [8]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset=get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)


sex                 : [b'male' b'male' b'male' b'male' b'female']
age                 : [26. 41. 48. 23. 26.]
n_siblings_spouses  : [2 2 1 2 0]
parch               : [0 0 0 1 0]
fare                : [ 8.663 14.108 76.729 11.5    7.925]
class               : [b'Third' b'Third' b'First' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'D' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'n' b'n' b'y']


In [9]:
SELECT_COLUMNS=['survived','age','n_siblings_spouses','class','deck','alone']
temp_dataset=get_dataset(train_file_path,select_columns=SELECT_COLUMNS)
show_batch(temp_dataset)

age                 : [40. 21.  3. 19. 36.]
n_siblings_spouses  : [1 0 1 0 0]
class               : [b'Third' b'Third' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'n' b'y' b'n' b'y' b'y']


In [11]:
SELECT_COLUMNS=['survived','age','n_siblings_spouses','parch','fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                           select_columns=SELECT_COLUMNS,
                           column_defaults=DEFAULTS)
show_batch(temp_dataset)

age                 : [25. 22. 19. 24. 20.]
n_siblings_spouses  : [1. 0. 0. 0. 1.]
parch               : [0. 2. 0. 0. 0.]
fare                : [ 7.775 49.5   10.171 49.504  7.925]


In [12]:
def pack(feautures, label):
    return tf.stack(list(feautures.values()), axis=-1),label

In [14]:
packed_dataset=temp_dataset.map(pack)

for feautures, labels in packed_dataset.take(1):
        print(feautures.numpy())
        print()
        print(labels.numpy())

[[ 28.      0.      0.      8.05 ]
 [ 14.      1.      2.    120.   ]
 [ 36.      0.      0.      7.496]
 [ 16.      0.      0.      7.75 ]
 [ 28.      0.      0.      7.312]]

[0 1 0 1 0]


In [15]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [11.   4.  30.5  2.  28. ]
n_siblings_spouses  : [0 1 0 3 0]
parch               : [0 1 0 1 0]
fare                : [18.788 16.7    8.05  21.075  8.458]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'G' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'n' b'y' b'n' b'y']


In [16]:
example_batch, labels_batch = next(iter(temp_dataset))

In [17]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names=names
    def __call__(self, features, labels):
        numeric_features=[features.pop(name) for name in self.names]
        numeric_features=[tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features=tf.stack(numeric_features, axis=-1)
        features['numeric']=numeric_features

        return features,labels

In [18]:
NUMERIC_FEATURES=['age', 'n_siblings_spouses', 'parch', 'fare']
packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data=raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [19]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
class               : [b'First' b'Third' b'Third' b'First' b'Third']
deck                : [b'A' b'unknown' b'unknown' b'C' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']
numeric             : [[ 71.      0.      0.     34.654]
 [ 24.      0.      0.      7.496]
 [ 20.5     0.      0.      7.25 ]
 [ 58.      0.      1.    153.462]
 [ 49.      0.      0.      0.   ]]


In [20]:
example_batch, labels_batch=next(iter(packed_train_data))

In [21]:
import pandas as pd
desc=pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [22]:
MEAN=np.array(desc.T['mean'])
STD=np.array(desc.T['std'])

In [23]:
def normalize_numeric_data(data,mean,std):
    # Center the data
    return (data-mean)/std

In [24]:
normalizer=functools.partial(normalize_numeric_data,mean=MEAN,std=STD)
numeric_column=tf.feature_column.numeric_column('numeric',normalizer_fn=normalizer,shape=[len(NUMERIC_FEATURES)])
numeric_columns=[numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x0000020180F87550>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [25]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 9.   ,  1.   ,  1.   , 15.9  ],
       [28.   ,  0.   ,  0.   ,  7.55 ],
       [29.   ,  0.   ,  0.   ,  9.483],
       [28.   ,  0.   ,  0.   ,  8.05 ],
       [14.   ,  0.   ,  0.   ,  7.854]], dtype=float32)>

In [26]:
numeric_layer=tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-1.649,  0.395,  0.782, -0.339],
       [-0.13 , -0.474, -0.479, -0.492],
       [-0.05 , -0.474, -0.479, -0.456],
       [-0.13 , -0.474, -0.479, -0.482],
       [-1.249, -0.474, -0.479, -0.486]], dtype=float32)

In [27]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [28]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [29]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [30]:
categorical_layer=tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [31]:
preprocessing_layer=tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [32]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -1.649  0.395
  0.782 -0.339  1.     0.   ]


In [34]:
model=tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(1),
])
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [35]:
train_data=packed_train_data.shuffle(500)
test_data=packed_test_data

In [36]:
model.fit(train_data,epochs=20)

Epoch 1/20
126/126 [==============================] - 1s 7ms/step - loss: 0.4940 - accuracy: 0.7544
Epoch 2/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4179 - accuracy: 0.8166
Epoch 3/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4141 - accuracy: 0.8230
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3964 - accuracy: 0.8214
Epoch 5/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3744 - accuracy: 0.8357
Epoch 6/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3644 - accuracy: 0.8437
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3733 - accuracy: 0.8357
Epoch 8/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3584 - accuracy: 0.8357
Epoch 9/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3467 - accuracy: 0.8469
Epoch 10/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3466 - accuracy: 0.8437

In [37]:
test_loss, test_accuracy=model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 1ms/step - loss: 0.4674 - accuracy: 0.8409


Test Loss 0.46743401885032654, Test Accuracy 0.8409090638160706


In [39]:
predictions=model.predict(test_data)

#SHOW some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    prediction=tf.sigmoid(prediction).numpy()
    print("Predicted survival: {:2%}".format(prediction[0]),
    " | Actual outcome: ",
    ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 8.111518%  | Actual outcome:  SURVIVED
Predicted survival: 99.846554%  | Actual outcome:  DIED
Predicted survival: 99.958771%  | Actual outcome:  SURVIVED
Predicted survival: 11.051155%  | Actual outcome:  DIED
Predicted survival: 94.404858%  | Actual outcome:  SURVIVED
